In [1]:
import numpy as np
import jax.numpy as jnp
from jax import grad, jit, jacfwd, vmap

from THB.bspline_funcs import *
from THB.core import *
from THB.jax_funcs import *
from THB.multilevel_spline_space import *

In [2]:
bs1 = BSpline(knotvector=np.array([0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1]), degree=2)
bs2 = BSpline(knotvector=np.array([0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1]), degree=2)
bs3 = BSpline(knotvector=np.array([0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1]), degree=2)

tp = TensorProduct([bs1, bs2, bs3])
num_levels = 3

h_space = Space(tp, num_levels=num_levels)
h_space.build_hierarchy_from_domain_sequence()

ac_cells = compute_active_cells_active_supp(h_space.cells, h_space.fns, h_space.degrees)
fn_coeffs = compute_refinement_operators(h_space.fns, h_space.Coeff, h_space.degrees)

params = generate_parametric_coordinates((25, 25, 25))
    
ac_cell_supp, num_supp = compute_active_span(params, h_space.knotvectors, h_space.cells, h_space.degrees, ac_cells)

# PHI = THB_basis_fns_tp_parallel(params, ac_cell_supp, fn_coeffs, h_space.sh_fns, h_space.knotvectors, h_space.degrees)


100%|██████████| 15625/15625 [00:00<00:00, 811209.59it/s]


In [3]:
new_PHI = THB_basis_fns(jnp.array(params), ac_cell_supp, num_supp, fn_coeffs, h_space.sh_fns, h_space.knotvectors, h_space.degrees)

1
2
3
4
5
(421875, 3, 3, 3) (12719, 22, 22, 22) (421875, 22, 22, 22)
6


In [4]:
old_PHI = THB_basis_fns_tp_parallel(params, ac_cell_supp, fn_coeffs, h_space.sh_fns, h_space.knotvectors, h_space.degrees)

In [5]:
jnp.sum(new_PHI[0:27])

Array(1.0000315, dtype=float32)

In [6]:
old_flat = old_PHI.reshape(-1, 1)
jnp.sum(old_flat[0:27])

Array(1.0000002, dtype=float32)

In [7]:
new_PHI

Array([9.9970025e-01, 9.9976256e-05, 2.4995004e-05, ..., 8.1919273e-03,
       1.3926423e-01, 2.6214761e-01], dtype=float32)

In [8]:
old_flat

Array([[9.9970025e-01],
       [9.9976256e-05],
       [1.2497502e-09],
       ...,
       [8.1919273e-03],
       [1.3926423e-01],
       [2.6214761e-01]], dtype=float32)